In [1]:
import sys
sys.path.append('../')

In [2]:
import numpy as np
import pandas as pd

from libreco.data import DatasetFeat
from libreco.algorithms import SVDpp, BPR
from libreco.evaluation import evaluate

from sklearn.model_selection import train_test_split

from tqdm import tqdm

from src import evaluation as eval

Instructions for updating:
non-resource variables are not supported in the long term


# Read Preprocessed Data 
Data was preprocessed during FunkSVD training

In [3]:
df = pd.read_csv('../data/df_with_emb_cleaned.csv')
df.head()

,user,item,label,Age,pca_dim_1,pca_dim_2,pca_dim_3,pca_dim_4,pca_dim_5,pca_dim_6,pca_dim_7,pca_dim_8,pca_dim_9,pca_dim_10,Year
0,67544.0,0440214009,7.0,30.0,0.098736,-0.016876,-0.064115,-0.043559,0.011505,0.062887,0.004183,0.003117,0.043744,0.009762,1993.0
1,67544.0,0688077080,8.0,30.0,-0.063320,-0.039060,0.001323,-0.025799,0.026691,0.008113,0.002605,-0.110809,0.049842,-0.035884,1989.0
2,219008.0,0679405135,4.0,60.0,0.031406,0.060715,0.081350,-0.115592,0.063185,0.093926,0.010901,0.028100,-0.013741,-0.079226,1996.0
3,219008.0,0446519723,7.0,60.0,0.150597,-0.103935,0.003405,0.022489,-0.018591,-0.136100,0.041277,0.053403,0.011589,0.037841,1995.0
4,219008.0,0140096361,6.0,60.0,-0.054594,0.001724,-0.063644,-0.011405,-0.014985,0.116361,0.030079,-0.026273,0.054377,0.051752,1987.0


In [4]:
df.columns

Index(['user', 'item', 'label', 'Age', 'pca_dim_1', 'pca_dim_2', 'pca_dim_3',
       'pca_dim_4', 'pca_dim_5', 'pca_dim_6', 'pca_dim_7', 'pca_dim_8',
       'pca_dim_9', 'pca_dim_10', 'Year'],
      dtype='object')

In [5]:
user_col = ['Age']
book_col = list(df.columns[4:])

# Train / Eval Split

In [6]:
train_df, eval_df = train_test_split(df, test_size=0.2, random_state=0)

In [7]:
eval_user_mask = eval_df['user'].isin(train_df['user'].unique())
eval_item_mask = eval_df['item'].isin(train_df['item'].unique())

eval_df = eval_df[eval_user_mask & eval_item_mask]

In [8]:
train_df.shape, eval_df.shape

((8195, 15), (496, 15))

In [9]:
train_data, data_info = DatasetFeat.build_trainset(
    train_df,
    user_col=user_col,
    item_col=book_col,
    dense_col=book_col+user_col,
)
eval_data = DatasetFeat.build_evalset(eval_df)

# Back-End Model: FunkSVD

In [10]:
funkSVD = SVDpp(
    task="rating",
    data_info=data_info,
    n_epochs=100,
    embed_size=128,
    lr=0.02,
    reg=0.01,
    seed=42
)

In [11]:
funkSVD.fit(
    train_data, 
    verbose=2, 
    neg_sampling=False, 
    eval_data=eval_data, 
    metrics=['rmse','mae']
)

Training start time: 2024-08-11 22:46:56


2024-08-11 22:46:57.065287: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2024-08-11 22:46:57.074330: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
train: 100%|██████████| 65/65 [00:00<00:00, 219.37it/s]


Epoch 1 elapsed: 0.315s
	 train_loss: 61.2066


eval_pointwise: 100%|██████████| 1/1 [00:00<00:00, 2576.35it/s]


	 eval rmse: 7.1241
	 eval mae: 6.9350


...

	 eval rmse: 5.8509
	 eval mae: 5.3331


train: 100%|██████████| 65/65 [00:00<00:00, 244.58it/s]


Epoch 100 elapsed: 0.267s
	 train_loss: 29.6377


eval_pointwise: 100%|██████████| 1/1 [00:00<00:00, 2236.96it/s]

	 eval rmse: 5.8549
	 eval mae: 5.3396


# Front-End Model: BPR

In [12]:
embedding_size = 64
n_epochs_bpr = 100
learning_rate = 0.01
batch_size = 256
random_seed = 42

In [13]:
# init BPR model
bpr = BPR(
    task="ranking",  
    data_info=data_info,
    embed_size=embedding_size,  
    n_epochs=n_epochs_bpr,  
    lr=learning_rate,  
    reg=None,  
    batch_size=batch_size,  
    num_neg=1,  
    use_tf=True,  
    seed=random_seed
)

In [14]:
# train BPR model
bpr.fit(
    train_data, 
    neg_sampling=True,
    verbose=2, 
    shuffle=True, 
    eval_data=eval_data, 
    metrics=["ndcg", "precision"]
)

Training start time: 2024-08-11 22:47:25


train: 100%|██████████| 33/33 [00:00<00:00, 316.32it/s]


Epoch 1 elapsed: 0.105s
	 train_loss: 0.6967


eval_listwise: 100%|██████████| 227/227 [00:00<00:00, 2714.01it/s]


	 eval ndcg@10: 0.0338
	 eval precision@10: 0.0066


...


	 eval precision@10: 0.0018


train: 100%|██████████| 33/33 [00:00<00:00, 132.62it/s]


Epoch 100 elapsed: 0.251s
	 train_loss: 0.0458


eval_listwise: 100%|██████████| 227/227 [00:00<00:00, 3249.96it/s]

	 eval ndcg@10: 0.0091
	 eval precision@10: 0.0018


# 2-Stage Recommender Evaluation

In [15]:
# get top 10 predictions for each user using FunkSVD
top_10_predictions = {}
for user_id in tqdm(eval_df['user'].unique()):
    top_10_predictions[user_id] = funkSVD.recommend_user(user_id, 10)[user_id]

# ranking the top 10 items using BPR
ranked_predictions = {}
for user_id, items in tqdm(top_10_predictions.items()):
    scores = {item: bpr.predict(user_id, item)[0] for item in items}
    ranked_predictions[user_id] = sorted(scores, key=scores.get, reverse=True)

100%|██████████| 227/227 [00:00<00:00, 4428.65it/s]


In [16]:
precisions = []
recalls = []
ndcgs = []
average_precisions = []

for user_id in tqdm(eval_df['user'].unique()):
    user_mask = (df['user'] == user_id)
    rating_mask = (df['label'] >= 4)

    ground_truth = df[user_mask & rating_mask]['item'].tolist()
    ranked_list = ranked_predictions[user_id]

    precisions.append(eval.precision_at_k(ranked_list, ground_truth, 10))
    recalls.append(eval.recall_at_k(ranked_list, ground_truth, 10))
    ndcgs.append(eval.ndcg_at_k(ranked_list, ground_truth, 10))
    average_precisions.append(eval.average_precision(ranked_list, ground_truth, m=10))

print(f'Precision at k=10: {np.mean(precisions)}')
print(f'Recall at k=10: {np.mean(recalls)}')
print(f'NDCG at k=10: {np.mean(ndcgs)}')
print(f'Average precision at m=10: {np.mean(average_precisions)}')

100%|██████████| 227/227 [00:00<00:00, 3548.82it/s]

Precision at k=10: 0.00616740088105727
Recall at k=10: 0.009251031029665392
NDCG at k=10: 0.008096682622718572
Average precision at m=10: 0.006420311516677155


# Model Analysis

Pros: 
- the task is splitted in candidate generation (which is done by predicting an item rating), and then ranking through listwise or pairwise item comparison
- candidate generation reduces the number of items to compare 
- different models can be used both for candidate generation, and ranking

Cons:
- bad candidate generation leads results in meaningless ranking
- problems with evaluation clarity (you need to evaluate both parts of a model)